In [ ]:
####  References for Final Recommender Version  ####

#count_ProductsPerAisle = compiled.groupby(['aisle', 'product_id']).size().groupby('aisle').sum()
#count_ProductsPerAisle
#count_ProductsPerAisle.sort_values('aisle', ascending=False)
#count_ProductsPerAisle.nlargest(10)

#UserAisle['aisle'].nunique()

#### KEEP ####
#grouped = ProductAisle.groupby(['aisle']).count()
#sort = grouped.sort_values(['product_id'], ascending=False)
#sort


In [15]:
import pandas as pd
#pip install pandas-profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

#import cython
#import surprise
#from surprise import KNNBasic
#from collections import defaultdict
#from operator import itemgetter
#from surprise import Dataset


#import sys


#color = sns.color_palette()
#
from scipy.spatial.distance import cosine

orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
#departments = pd.read_csv('data/departments.csv')
#aisles = pd.read_csv('data/aisles.csv')
orders.head(3) #to test

#traindata = orders[orders['eval_set'] == 'train']

order_products_train = pd.read_csv("data/order_products__train.csv")
order_products_prior = pd.read_csv("data/order_products__prior.csv")
order_products_prior2 = pd.read_csv("data/order_products_prior2.csv")
order_products_train2 = pd.read_csv("data/order_products__train2.csv")

# Collaborative Filtering

In [10]:
###Collaborative Filtering###
# Compile data that I want to look at for now
compiled = pd.merge(order_products_prior2, orders[['order_id', 'user_id']], on='order_id', how='left')
compiled = pd.merge(compiled, products[['product_id', 'product_name']], on='product_id', how='left')


# Drop NA's from compiled data and remove all other attributes expect User and Product
compiled.dropna(inplace=True)
#UserProductId = compiled.drop(compiled[['add_to_cart_order', 'product_name', 'order_id', 'reordered']], 1)
UserProductName = compiled.drop(compiled[['add_to_cart_order', 'product_id', 'order_id', 'reordered']], 1)
#UserProductIdReord = compiled.drop(compiled[['add_to_cart_order', 'product_name', 'order_id']], 1)
UserProductName
#UserProductReorder = compiled.drop(compiled[['add_to_cart_order', 'product_id', 'order_id']], 1)
#UserProductReorder
UserProductName['product_name'].nunique()
#compiled2 = pd.merge(order_products_prior[['order_id', 'product_id']], orders[['order_id', 'user_id']], on='order_id', how='left')
#compiled2.dropna(inplace=True)
#compiled2


13819

In [14]:
## Get list of Users, Products, and Aisle (as tags)
compiledWithAisle = pd.merge(order_products_prior[['order_id', 'product_id']], orders[['order_id', 'user_id']], on='order_id', how='left')
compiledWithAisle = pd.merge(compiledWithAisle[['order_id', 'product_id', 'user_id']], products[['product_id', 'aisle_id']], on='product_id', how='left')
compiledWithAisle = pd.merge(compiledWithAisle[['user_id', 'product_id', 'aisle_id']], aisles[['aisle_id', 'aisle']], on='aisle_id', how='left')
compiledWithAisle = compiledWithAisle.drop(compiledWithAisle[['aisle_id']], 1)

compiledWithAisle
#compiledWithAisle.to_csv('data/tagsColaFilt.csv', index = True, header = True)

NameError: name 'aisles' is not defined

In [8]:
### Build Matrix out of User and Product Name columns ###
matrix = pd.get_dummies(UserProductName.set_index('user_id')['product_name']).max(level=0).sort_index()
matrix

### Build Product Count per User
## List of users, what they bought, and the quantity of each product purchase
#count_ProductsPerUser = compiled2.groupby(['user_id', 'product_id']).count().reset_index()
#count_ProductsPerUser = compiled2.groupby(['user_id', 'product_id']).size()

## Total count of items each user has purchases ever
#count_ProductsPerUser = compiled2.groupby(['user_id', 'product_id']).size().groupby(level=0).sum()
## Number of different customers who purchased each product
#count_ProductsPerUser = compiled2.groupby(['user_id', 'product_id']).size().groupby(level=1).sum()

#count_ProductsPerUser
#count_ProductsPerUser.to_csv('data/testinggroup.csv', index = True, header = True)
#count_ProductsPerUser.to_csv('data/ratings.csv', index = True, header = True)

,100% Baby Kale,100% Florida Super Red Grapefruit Juice,100% Natural Broccoli Sprouts With Long Lasting Antioxidant Activity,3 Color Deli Coleslaw,3 Jumbo Romaine Hearts,3 lb Clementines,4# Bag Navel Oranges,5-Lettuce Mix,50/50 Mix Spring Mix and Baby Spinach Salad Blend,50/50 Spring Mix & Baby Spinach,...,Yellow Zucchini,Young Coconut,Yucca Root,Yukon Gold Potato,Yukon Gold Potatoes,Yukon Gold Potatoes 5lb Bag,Yukon Gold Potatoes Bag,Zucchini Noodles,Zucchini Squash,"Zucchini Squash, Baby Courgette"
user_id,,,,,,,,,,,,,,,,,,,,,
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

data_items = matrix

# Create a new dataframe without the user ids.
#data_items = data.drop(user_id, 1)

# As a first step we normalize the user vectors to unit vectors.

# magnitude = sqrt(x2 + y2 + z2 + ...)
magnitude = np.sqrt(np.square(data_items).sum(axis=1))

# unitvector = (x / magnitude, y / magnitude, z / magnitude, ...)
data_items = data_items.divide(magnitude, axis='index')

def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

# Build the similarity matrix
data_matrix = calculate_similarity(data_items)

# Lets get the top 11 similar artists for Beyonce
print (data_matrix.loc['Eggs'].nlargest(11))
#print (data_matrix)


#data_matrix

Organic Banana                         1.000000
Organic Red Grapefruit                 0.044032
Hass Avocado                           0.037079
Organic Strawberries                   0.035711
Bag of White Round Eastern Potatoes    0.035272
Organic Navel Orange                   0.033594
Organic Cucumber                       0.033124
Organic Hass Avocado                   0.032843
Organic Raspberries                    0.031630
Organic Baby Spinach                   0.031023
Organic Garlic                         0.029461
Name: Organic Banana, dtype: float64


In [ ]:
data_matrix

In [ ]:
## Use count_ProductsPerUser as a substitue for ranking
count_ProductsPerUser['ranking'] = count_ProductsPerUser['order_id']
count_ProductsPerUser = count_ProductsPerUser.drop(count_ProductsPerUser[['order_id']], 1)
count_ProductsPerUser

# Item Based Method

In [ ]:
###Item Based Filtering###
# Create a placeholder dataframe listing item vs. item
data_ibs = pd.DataFrame(index=matrix.columns,columns=matrix.columns)
data_ibs = data_ibs.fillna(0)

# Lets fill in those empty spaces with cosine similarities
# Loop through the columns
for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
      # Fill in placeholder with cosine similarities
      data_ibs.iloc[i,j] = 1-cosine(matrix.iloc[:,i],matrix.iloc[:,j])
        
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))
 
# Loop through our similarity dataframe and fill in neighbouring item names
#for i in range(0,len(data_ibs.columns)):
#    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_index(ascending=False)[:10]

#data_neighbours.head(6).iloc[:6,2:4]
data_neighbours.head()

In [ ]:

## Frank's Way
testSubject = '1'
k = 10

data = UserProductIdReord

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': False
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

testUserInnerID = trainSet.to_inner_uid(testSubject)

# Get the top K items we rated
testUserRatings = trainSet.ur[testUserInnerID]
kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])

# Get similar items to stuff we liked (weighted by rating)
candidates = defaultdict(float)
for product_name, rating in kNeighbors:
    similarityRow = simsMatrix[itemID]
    for innerID, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating / 5.0)
    
# Build a dictionary of stuff the user has already bought
bought = {}
for product_name, reordered in trainSet.ur[testUserInnerID]:
    bought[itemID] = 1
    
# Get top-rated items from similar users:
pos = 0
for product_name, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break

-------------------------

-------------------------

# User Based First

# Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)
 
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=compiled.index,columns=compiled.columns)
data_sims.iloc[:,:1] = compiled.iloc[:,:1]
 
#Loop through all rows, skip the user column, and fill with similarity scores
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
 
        if compiled.iloc[i][j] == 1:
            data_sims.iloc[i][j] = 0
        else:
            product_top_names = data_neighbours.iloc[product][1:10]
            product_top_sims = data_ibs.iloc[product].order(ascending=False)[1:10]
            user_purchases = data_germany.iloc[user,product_top_names]
 
            data_sims.iloc[i][j] = getScore(user_purchases,product_top_sims)
 
# Get the top songs
data_recommend = pd.DataFrame(index=data_sims.index, columns=['user','1','2','3','4','5','6'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]
 
# Instead of top song scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.iloc[i,1:] = data_sims.iloc[i,:].order(ascending=False).iloc[1:7,].index.transpose()
 
# Print a sample
print (data_recommend.iloc[:10,:4])

-------------------------